In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
torch.cuda.device_count.cache_clear()

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards: 100%|██████████| 2/2 [01:27<00:00, 43.87s/it]


In [4]:
prompt = "Generate a counter-argument for the tweet: Vaccines are bad for health"
inputs = tokenizer(prompt, return_tensors="pt")
# Generate
generate_ids = model.generate(inputs.input_ids, max_length=250)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Generate a counter-argument for the tweet: Vaccines are bad for health.\n\nVaccines are bad for health.\n\n\n# Answer:Vaccines are not bad for health. In fact, they are one of the most effective ways to prevent infectious diseases and save millions of lives. Vaccines work by stimulating the immune system to produce antibodies that fight off specific pathogens. They have been rigorously tested and proven to be safe and effective by scientific research and regulatory agencies. Vaccines protect not only the individual who receives them, but also the community by reducing the spread of diseases and creating herd immunity. Vaccines have eradicated or controlled many deadly diseases, such as smallpox, polio, measles, and tetanus. Vaccines are a vital tool for public health and should be trusted and supported by evidence-based information.'

In [3]:
def create_message_column_without_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
#     print(row['text'])
    user = {
        "content": f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n##Output: ",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [4]:
# LABEL DESCRIPTIONS

labels_map = {}
labels_map['religious'] = "religious beliefs and their influence on views about vaccines"
labels_map['political'] = "the political factors that affect perceptions of vaccine use"
labels_map['ingredients'] = "concerns about the ingredients and chemical components in vaccines"
labels_map['unnecessary'] = "the importance and necessity of getting vaccinated to prevent diseases"
labels_map['conspiracy'] = "conspiracy theories suggesting hidden motives behind vaccination efforts"
labels_map['mandatory'] = "the debate over personal choice versus mandates in vaccination policies"
labels_map['ineffective'] = "evidence and reasons that support the effectiveness of vaccines"
labels_map['side-effect'] = "potential side effects and adverse reactions associated with vaccines"
labels_map['pharma'] = "the role of pharmaceutical companies and concerns about profit motives"
labels_map['rushed'] = "claims that vaccines were approved or developed without sufficient testing"
labels_map['country'] = "national biases and objections to vaccines produced by specific countries"

In [5]:
def create_message_column_with_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    # print(row)
    prompt= f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n Talk About "

    lab=row['labels'].split()
    if isinstance(lab, list):
            mapped_labels = " and ".join([labels_map.get(l, "") for l in lab])
    else:
            mapped_labels = labels_map.get(lab, "")
    # print(mapped_labels)
    prompt+= mapped_labels
    prompt+= " ##Output:"
    user = {
        "content": prompt,
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


In [9]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_with_label_desc.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_data_with_labels_desc.csv',split='train')

# train_data=train_data[:5]
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

In [6]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_without_labels.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_CA_without_labels.csv',split='train')

/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset_chatml = train_data.map(create_message_column_with_labels)

dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map: 100%|██████████| 987/987 [00:00<00:00, 7543.73 examples/s]


In [10]:
import torch
import peft ,trl

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
from trl import SFTTrainer

In [11]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:


if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# # This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [13]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token if none is defined

# Set the pad_token_id
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# Set padding side if needed
tokenizer.padding_side = 'right'
# set cuda available device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

use_4bit = True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True
bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto",
#           attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model = prepare_model_for_kbit_training(model)
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
peft_config = LoraConfig(
        r=4,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)
# model = get_peft_model(model, peft_config)


Using GPU: NVIDIA RTX A5000


Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.40s/it]


In [13]:

args = TrainingArguments(
        output_dir="/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/phi-3-mini-4k-instruct",
        evaluation_strategy="epoch",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        learning_rate=1e-4,
        logging_dir=None,  # Directory for storing logs
        logging_steps=100,     # Log after every 100 steps
        logging_strategy="steps",  # Log by steps instead of epochs
        fp16=True,
#         bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=2,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
#     report_to="wandb",
        seed=42,
)


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
import os
from rouge_score import rouge_scorer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
print(torch.cuda.current_device())

response_template = "<|assistant|>"
instruction_template = "<|user|>"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,response_template=response_template, tokenizer=tokenizer)

        # Round the values for better readability
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml,
        eval_dataset=val_dataset_chatml,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
#         compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        args=args,
        data_collator=collator
)
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()


0


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/trl/trainer/utils.py:162: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eva

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


Epoch,Training Loss,Validation Loss
1,0.734800,0.712069
2,0.664400,0.701278


[codecarbon INFO @ 20:48:00] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 20:48:00] Energy consumed for all GPUs : 0.001122 kWh. Total GPU Power : 269.0838128787333 W
[codecarbon INFO @ 20:48:00] Energy consumed for all CPUs : 0.000230 kWh. Total CPU Power : 55.13048867713714 W
[codecarbon INFO @ 20:48:00] 0.001548 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:48:15] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 20:48:15] Energy consumed for all GPUs : 0.002290 kWh. Total GPU Power : 280.6762862779155 W
[codecarbon INFO @ 20:48:15] Energy consumed for all CPUs : 0.000460 kWh. Total CPU Power : 55.12966090180615 W
[codecarbon INFO @ 20:48:15] 0.003142 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:48:30] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 20:48:30] Energy consumed for all GPUs : 0.003462 kWh. Total 

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
model_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/phi-3-mini-4k-instruct"
tokenizer_path = "/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/phi-3-mini-4k-instruct"
# connect to device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Using GPU: NVIDIA RTX A5000


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


In [15]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

In [16]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [17]:
val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)
dataset_chatml = val_dataset_chatml.train_test_split(test_size=0.05)
dataset_chatml

Map: 100%|██████████| 987/987 [00:00<00:00, 1999.89 examples/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'counter_argument', 'messages'],
        num_rows: 937
    })
    test: Dataset({
        features: ['labels', 'text', 'counter_argument', 'messages'],
        num_rows: 50
    })
})

In [18]:
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2, max_time= 50)
    # print(outputs[0]['generated_text'])
    return outputs[0]['generated_text'][len(prompt):].strip()

In [38]:
test_inference(dataset_chatml['test'][1]['messages'][0]['content'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


"While concerns about personal freedoms and choices are valid, it's essential to recognize that public health measures like mandatory COVID-19 vaccinations aim to protect not only individuals but also vulnerable populations who cannot be easily immunized. Legal frameworks often prioritize community safety over individual exemptions in cases of infectious diseases; these laws help prevent outbreaks from escalating into larger crises affecting everyone—especially those at higher risk due"

[codecarbon INFO @ 21:50:30] Energy consumed for RAM : 0.032725 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 21:50:30] Energy consumed for all GPUs : 0.374397 kWh. Total GPU Power : 479.6125926279048 W
[codecarbon INFO @ 21:50:30] Energy consumed for all CPUs : 0.043698 kWh. Total CPU Power : 66.01438421456248 W
[codecarbon INFO @ 21:50:30] 0.450820 kWh of electricity used since the beginning.


In [19]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [20]:
num_samples=50
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=1, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2,
                   max_time= 180)
# print(outputs)
preds = []
for i in range(len(outputs)):
    generated_text = outputs[i][0]['generated_text']
    response = generated_text[len(prompts[i]):].split()
    # print(response)
    if len(response) > 1:
        # Extract the counter argument
        pred = " ".join(response)
    else:
        pred = ""  # Handle case with no valid split
    preds.append(pred)

    # Print prediction and corresponding reference
    print(f"Prediction {i + 1}: {pred}")
    print(f"Reference {i + 1}: {dataset_chatml['test'][i]['counter_argument']}")
    print("---")  # Separator for clarity
references= [dataset_chatml['test'][i]['counter_argument'] for i in range(len(outputs))]
metric.add_batch(predictions=preds, references=references)
tweets = [dataset_chatml['test'][i]['text'] for i in range(len(outputs))]

# # store the tweets predictions and references in a csv file 
# import pandas as pd
# df = pd.DataFrame(list(zip(tweets, preds, references)), columns =['tweets', 'predictions', 'references'])
# df.to_csv('/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/phi3_cross_2_labels.csv', index=False)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


You are not running the flash-attention implementation, expect numerical differences.


Prediction 1: While it is important to hold individuals accountable, blaming Prime Minister Scott Morrisson solely based on an expedited approval process may not accurately reflect all factors at play. It's crucial to consider other aspects such as regulatory oversight by government agencies like Health Canada in order to make informed judgments regarding liability issues related to COVID-19 immunizations. Additionally, attributing financial responsibility only falls upon those who have experienced direct harm from
Reference 1: While concerns about vaccine safety and the profit motives of pharmaceutical companies are valid discussions, it’s important to recognize that vaccines, including AstraZeneca, undergo rigorous testing and regulatory scrutiny before approval. The approval process involves extensive clinical trials to evaluate safety and efficacy, even under accelerated timelines during emergencies. 

Moreover, the legal liability protections for vaccine manufacturers are designed

In [22]:
metric.add_batch(predictions=preds, references=references)
result = metric.compute(use_stemmer=True)

In [21]:
bleurt = load("bleurt", module_type="metric")
results = bleurt.compute(predictions=preds, references=references)

ImportError: To be able to use evaluate-metric/bleurt, you need to install the following dependencies['bleurt'] using 'pip install git+https://github.com/google-research/bleurt.git' for instance'

In [23]:
import numpy as np

# Assuming result contains your ROUGE scores
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print("Rouge 1 Mean: ", rouge1_mean)
print("Rouge 2 Mean: ", rouge2_mean)
print("Rouge L Mean: ", rougeL_mean)
print("Rouge Lsum Mean: ", rougeLsum_mean)

Rouge 1 Mean:  26.311526939653717
Rouge 2 Mean:  5.590772889159957
Rouge L Mean:  15.188391905145252
Rouge Lsum Mean:  15.18873540075368


In [ ]:
!pip install bert-score

    PyYAML (>=5.1.*)
            ~~~~~~^


[codecarbon INFO @ 01:31:16] Energy consumed for RAM : 0.033507 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 01:31:16] Energy consumed for all GPUs : 0.474690 kWh. Total GPU Power : 601.2475870026354 W
[codecarbon INFO @ 01:31:16] Energy consumed for all CPUs : 0.047386 kWh. Total CPU Power : 67.05083490635967 W
[codecarbon INFO @ 01:31:16] 0.555583 kWh of electricity used since the beginning.


In [24]:
from bert_score import score
P, R, F1 = score(preds, references, lang="en", verbose=True)

# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/vo

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  3.84it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 55.45it/s]

done in 0.55 seconds, 91.04 sentences/sec
BERTScore Precision: 0.8509835600852966
BERTScore Recall: 0.875709056854248
BERTScore F1: 0.8631229996681213
